# Chess Engine Board Visualization

Experiments and tests for the BarelyBlue chess engine.

## Setup
Make sure you have installed all dependencies:
```bash
pip install -r requirements
```

In [ ]:
# Import required libraries
import chess
import chess.svg
from IPython.display import display, SVG, clear_output
import matplotlib.pyplot as plt
import numpy as np
import time

# Import chess engine components
import sys
sys.path.append('..')

from chess_engine.evaluation import ClassicalEvaluator
from chess_engine.search import find_best_move, TranspositionTable
from chess_engine.board import board_to_tensor

print("✓ Imports successful")

## Basic Board Display

Display a chess board using SVG rendering.

In [ ]:
def display_board(board, size=400):
    """
    Display a chess board as SVG.
    
    Args:
        board: chess.Board object
        size: Board size in pixels
    """
    svg = chess.svg.board(board, size=size)
    display(SVG(svg))

# Display starting position
board = chess.Board()
print("Starting Position:")
display_board(board)

## Position Evaluation

Evaluate a position using the classical evaluator.

In [ ]:
def analyze_position(board, depth=5, verbose=True):
    """
    Analyze a position and display evaluation.
    
    Args:
        board: chess.Board object
        depth: Search depth
        verbose: Print detailed info
    
    Returns:
        Tuple of (best_move, score)
    """
    evaluator = ClassicalEvaluator()
    tt = TranspositionTable(max_size=1000000)
    
    if verbose:
        print(f"Analyzing position at depth {depth}...")
    
    start_time = time.time()
    best_move, score = find_best_move(board, depth, evaluator, tt)
    elapsed = time.time() - start_time
    
    if verbose:
        print(f"\nAnalysis Results:")
        print(f"  Best Move: {best_move.uci()} ({board.san(best_move)})")
        print(f"  Evaluation: {score:.2f} centipawns")
        print(f"  Time: {elapsed:.2f} seconds")
        print(f"  TT Stats: {tt.get_stats()}")
    
    return best_move, score

# Analyze starting position
board = chess.Board()
best_move, score = analyze_position(board, depth=4)

# Display board with arrow showing best move
svg = chess.svg.board(board, arrows=[chess.svg.Arrow(best_move.from_square, best_move.to_square)])
display(SVG(svg))

## Interactive Game Play

Play a game by making moves interactively.

In [ ]:
def play_move(board, move_str):
    """
    Play a move on the board.
    
    Args:
        board: chess.Board object
        move_str: Move in SAN (e.g., 'e4') or UCI (e.g., 'e2e4') format
    
    Returns:
        True if move was legal, False otherwise
    """
    try:
        # Try parsing as SAN first
        move = board.parse_san(move_str)
    except ValueError:
        try:
            # Try parsing as UCI
            move = chess.Move.from_uci(move_str)
            if move not in board.legal_moves:
                print(f"Illegal move: {move_str}")
                return False
        except ValueError:
            print(f"Invalid move format: {move_str}")
            return False
    
    board.push(move)
    return True

# Example:
board = chess.Board()
moves = ['e4', 'e5', 'Nf3', 'Nc6']

for move in moves:
    play_move(board, move)
    print(f"Played: {move}")

display_board(board)

## Engine vs. Engine Game

Watch two engines play against each other.

In [ ]:
def engine_vs_engine(depth_white=4, depth_black=4, max_moves=50, display_boards=True):
    """
    Play an engine vs. engine game.
    
    Args:
        depth_white: Search depth for White
        depth_black: Search depth for Black
        max_moves: Maximum number of moves before draw
        display_boards: Show board after each move
    
    Returns:
        Tuple of (board, evaluations, moves)
    """
    board = chess.Board()
    evaluator = ClassicalEvaluator()
    tt = TranspositionTable(max_size=10000000)
    
    evaluations = []  # Track evaluation over time
    moves_played = []
    
    move_count = 0
    
    while not board.is_game_over() and move_count < max_moves:
        # Determine depth for current player
        depth = depth_white if board.turn == chess.WHITE else depth_black
        
        # Find best move
        best_move, score = find_best_move(board, depth, evaluator, tt)
        
        # Record evaluation and move
        evaluations.append(score)
        moves_played.append(board.san(best_move))
        
        # Make move
        board.push(best_move)
        move_count += 1
        
        # Display
        if display_boards:
            clear_output(wait=True)
            print(f"Move {move_count}: {moves_played[-1]} (eval: {score:.2f})")
            display_board(board, size=400)
            time.sleep(0.5)  # Pause to see each move
    
    # Game result
    print(f"\nGame Over: {board.result()}")
    print(f"Moves played: {' '.join(moves_played)}")
    
    return board, evaluations, moves_played

# Play a quick game
board, evals, moves = engine_vs_engine(depth_white=3, depth_black=3, max_moves=20, display_boards=False)
print(f"Final position:")
display_board(board)

## Evaluation Plot

Plot how the evaluation changes over the course of a game.

In [ ]:
def plot_evaluation(evaluations, title="Evaluation Over Time"):
    """
    Plot evaluation curve.
    
    Args:
        evaluations: List of evaluation scores
        title: Plot title
    """
    plt.figure(figsize=(12, 6))
    plt.plot(evaluations, linewidth=2)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    plt.xlabel('Move Number')
    plt.ylabel('Evaluation (centipawns)')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot evaluations from the game above
if evals:
    plot_evaluation(evals, "Engine vs. Engine Game")

## Board Tensor Visualization

Visualize the 12-channel tensor representation used for neural networks.

In [ ]:
def visualize_tensor(board, piece_type=chess.PAWN, color=chess.WHITE):
    """
    Visualize a single channel of the board tensor.
    
    Args:
        board: chess.Board object
        piece_type: chess.PAWN, chess.KNIGHT, etc.
        color: chess.WHITE or chess.BLACK
    """
    tensor = board_to_tensor(board)
    
    # Get channel index
    channel_map = {
        (chess.PAWN, chess.WHITE): 0,
        (chess.KNIGHT, chess.WHITE): 1,
        (chess.BISHOP, chess.WHITE): 2,
        (chess.ROOK, chess.WHITE): 3,
        (chess.QUEEN, chess.WHITE): 4,
        (chess.KING, chess.WHITE): 5,
        (chess.PAWN, chess.BLACK): 6,
        (chess.KNIGHT, chess.BLACK): 7,
        (chess.BISHOP, chess.BLACK): 8,
        (chess.ROOK, chess.BLACK): 9,
        (chess.QUEEN, chess.BLACK): 10,
        (chess.KING, chess.BLACK): 11,
    }
    
    channel = channel_map[(piece_type, color)]
    
    # Plot channel
    piece_names = {chess.PAWN: 'Pawn', chess.KNIGHT: 'Knight', chess.BISHOP: 'Bishop',
                   chess.ROOK: 'Rook', chess.QUEEN: 'Queen', chess.KING: 'King'}
    color_name = 'White' if color == chess.WHITE else 'Black'
    
    plt.figure(figsize=(6, 6))
    plt.imshow(tensor[channel], cmap='Blues', vmin=0, vmax=1, interpolation='nearest')
    # Add gridlines to show discrete squares
    ax = plt.gca()
    ax.set_xticks(np.arange(-0.5, 8, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 8, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)
    plt.title(f"{color_name} {piece_names[piece_type]} Channel")
    plt.xticks(range(8), ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    plt.yticks(range(8), ['8', '7', '6', '5', '4', '3', '2', '1'])
    plt.tight_layout()
    plt.show()

# Visualize white pawns
board = chess.Board()
visualize_tensor(board, chess.PAWN, chess.WHITE)

## Test Position Analysis

Analyze tactical test positions (e.g., Bratko-Kopec).

In [ ]:
# Example: Bratko-Kopec position #1
# Black to move and win with Qd1+
fen = "1k1r4/pp1b1R2/3q2pp/4p3/2B5/4Q3/PPP2B2/2K5 b - - 0 1"
board = chess.Board(fen)

print("Bratko-Kopec Position #1")
print("Black to move and win")
print(f"FEN: {fen}")
display_board(board)

# Analyze
best_move, score = analyze_position(board, depth=5)

# Show best move
svg = chess.svg.board(
    board,
    arrows=[chess.svg.Arrow(best_move.from_square, best_move.to_square, color='green')]
)
print("\nEngine's suggested move:")
display(SVG(svg))

## Performance Testing

Test search performance at various depths.

In [ ]:
def benchmark_search(board, max_depth=6):
    """
    Benchmark search performance at increasing depths.
    
    Args:
        board: chess.Board object
        max_depth: Maximum depth to test
    """
    evaluator = ClassicalEvaluator()
    tt = TranspositionTable(max_size=10000000)
    
    depths = []
    times = []
    
    print("Benchmarking search performance...\n")
    print(f"{'Depth':<10} {'Time (s)':<15} {'Best Move':<15} {'Score'}")
    print("-" * 60)
    
    for depth in range(1, max_depth + 1):
        start = time.time()
        best_move, score = find_best_move(board, depth, evaluator, tt)
        elapsed = time.time() - start
        
        depths.append(depth)
        times.append(elapsed)
        
        print(f"{depth:<10} {elapsed:<15.3f} {best_move.uci():<15} {score:.2f}")
    
    # Plot performance curve
    plt.figure(figsize=(10, 6))
    plt.plot(depths, times, marker='o', linewidth=2, markersize=8)
    plt.xlabel('Search Depth')
    plt.ylabel('Time (seconds)')
    plt.title('Search Performance vs. Depth')
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Log scale for exponential growth
    plt.tight_layout()
    plt.show()

# Benchmark on starting position
board = chess.Board()
benchmark_search(board, max_depth=5)

## Custom Position Setup

Set up custom positions for testing.

In [ ]:
# Enter FEN position here
custom_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"  # Starting position

board = chess.Board(custom_fen)
display_board(board)

# Analyze
best_move, score = analyze_position(board, depth=4)

## Resources

- python-chess docs: https://python-chess.readthedocs.io/
- Chess Programming Wiki: https://www.chessprogramming.org/